In [2]:
# Import Libraries

import tkinter as tk
import sqlite3

# Connect to the database
conn = sqlite3.connect('rrs.db')
c = conn.cursor()

# Create the GUI
root = tk.Tk()
root.title('Railway Reservation System')
root.geometry('800x600')
root.config(bg='#f0f0f0')

# Query 1: Retrieve all trains booked by a passenger
def query1():
    fname = fname_entry.get()
    lname = lname_entry.get()
    query = f"""SELECT  P.First_Name, P.Last_Name, T.Train_Name, 
                        T.Train_Number , B.Reservation_Status 
                FROM    Passenger P, Train T, Booked B 
                WHERE   P.First_Name = '{fname}' 
                    AND P.Last_Name = '{lname}'
                    AND B.Passenger_SSN = P.SSN
                    AND B.Reservation_Status = 'Booked' 
                    AND B.Train_Number = T.Train_Number ;"""
    execute_query(query)

# Query 2: Retrieve passengers with confirmed tickets on a given day
def query2():
    day = day_entry.get()
    query = f""" SELECT  P.First_Name, P.Last_Name
                 FROM    Passenger P, Booked B, Train T
                 WHERE   B.Reservation_Status = 'Booked'
                    AND  B.Passenger_SSN = P.SSN
                    AND  B.Train_Number = T.Train_Number
                    AND  T.Available_on like '%{day}%';"""
    execute_query(query)

# Query 3: Retrieve train and passenger information for passengers between ages 50 and 60
def query3():
    age_range_min =  age_range_min_entry.get()
    age_range_max =  age_range_max_entry.get()
    query = f"""SELECT T.Train_Number, T.Train_Name, T.Source_Station,  
                       T.Destination_Station, P.First_Name, P.Last_Name,    
                       Cast((Julianday('now')-Julianday(P.Birth_Date)) / 365 AS int) as Age,
                       P.Address, P.City, P.County, B.Reservation_Status, B.Ticket_Type
                FROM   Passenger P, Train T, Booked B
                WHERE  Age BETWEEN {age_range_min} and {age_range_max}
                   AND P.SSN = B.Passenger_SSN
                   AND B.Train_Number = T.Train_Number;"""
    execute_query(query)

# Query 4: List all train names along with the count of passengers
def query4():
    query = f"""SELECT  T.Train_Name, COUNT(*) AS Passenger_Count 
                FROM    Train T, Booked B 
                WHERE   B.Train_Number = T.Train_Number 
                    AND B.Reservation_Status = 'Booked'
                GROUP BY B.Train_Number;"""
    execute_query(query)

# Query 5: Retrieve passengers information travelling in that train
def query5():
    train_name = train_name_entry.get()
    query = f"""SELECT  P.First_Name, P.Last_Name, P.SSN  
                FROM    Booked B, Train T, Passenger P  
                WHERE   T.Train_Name = '{train_name}'
                    AND B.Train_Number = T.Train_Number 
                    AND B.Reservation_Status = 'Booked'
                    AND B.Passenger_SSN = P.SSN;"""
    execute_query(query)

# Query 6: Passenger enter SSN, train number and ticket type to cancel a train.
#   The next passenger in the waitlist change to booked status:
def query6():
    # Connect to the database:
    conn = sqlite3.connect('rrs.db')
    c = conn.cursor()
    
    # Get the SSN,train number, and ticket type of the passenger that cancel the ticket.
    p_SSN = p_SSN_entry.get()
    Train_nu = Train_nu_entry.get()
    Ticket_type = Ticket_type_entry.get()
    
    # Delete passenger record from Booked table:
    query_a = f"""DELETE FROM Booked
                 WHERE Passenger_SSN = {p_SSN};"""
    c.execute(query_a)
    conn.commit()
    
    # Get the name of the next passenger that have the same train number and ticket type in the waitlist:
    query_b= f"""SELECT P.First_Name, P.Last_Name
                 FROM   Booked Bk, Passenger P
                 WHERE  Bk.Passenger_SSN = (SELECT B.Passenger_SSN 
                                            FROM Booked B
                                            WHERE B.Train_Number ={Train_nu}
                                                AND B.Ticket_Type ='{Ticket_type}'
                                                AND B.Reservation_Status = 'WaitList'
                                            LIMIT 1)
                 AND Bk.Passenger_SSN = P.SSN;"""
    c.execute(query_b)
    conn.commit()
    rows = c.fetchall()
    
    # Print out the name of next passenger in the waitlist:
    result_text.delete(1.0, tk.END)
    if len(rows)>0:
        result_text.insert(tk.END, 'Cancel success!'+'\n')
        for row in rows:
            result_text.insert(tk.END, row)
        result_text.insert(tk.END, '  has Booked!')
    else:
        result_text.insert(tk.END, 'Cancel success! No one else in the waitlist')
    
    # Update the next passenger that have the same train number and ticket type in the waitlist to Booked:
    query_c = f"""UPDATE   Booked
                  SET      Reservation_Status = 'Booked'
                  WHERE    Passenger_SSN = (SELECT B.Passenger_SSN 
                                            FROM   Booked B
                                            WHERE  B.Train_Number = {Train_nu}
                                                AND B.Ticket_Type = '{Ticket_type}'
                                                AND B.Reservation_Status = 'WaitList'
                                            LIMIT 1);"""
        
    c.execute(query_c)
    conn.commit()  
    conn.close()
    
# Execute query (for query 1 to 5) and get restults function:
def execute_query(query):
    
    c.execute(query)
    rows = c.fetchall()
    
    result_text.delete(1.0, tk.END)
    result_count_text.config(text='')
    if len(rows)>0:
        headers = [description[0] for description in c.description]
        result_text.insert(tk.END, '\n'.join([str(header).rjust(25) for header in headers])+'\n\n')
        for row in rows:
            result_text.insert(tk.END, '\n'.join([str(r).rjust(25) for r in row])+'\n')
        result_count_text.config(text=f'{len(rows)} rows returned')
    else:
        result_text.insert(tk.END, 'No results found.')

    
# Create widgets and arrange the widgets using grid geometry manager:

fname_label = tk.Label(root, text='First Name').grid(row=0, column=0)
fname_entry = tk.Entry(root)
fname_entry.grid(row=0, column=1)

lname_label = tk.Label(root, text='Last Name').grid(row=1, column=0)
lname_entry = tk.Entry(root)
lname_entry.grid(row=1, column=1)

day_label = tk.Label(root, text='Day (Day of the Week)').grid(row=3, column=0)
day_entry = tk.Entry(root)
day_entry.grid(row=3, column=1)

age_range_min_label = tk.Label(root, text='Minimum Age').grid(row=5, column=0)
age_range_min_entry = tk.Entry(root)
age_range_min_entry.grid(row=5, column=1)

age_range_max_label = tk.Label(root, text='Maximum Age').grid(row=6, column=0)
age_range_max_entry = tk.Entry(root)
age_range_max_entry.grid(row=6, column=1)

train_name_label = tk.Label(root, text='Train Name').grid(row=9, column=0)
train_name_entry = tk.Entry(root)
train_name_entry.grid(row=9, column=1)

p_SSN_label = tk.Label(root, text="Passenger SSN:").grid(row=13, column=0)
p_SSN_entry = tk.Entry(root)
p_SSN_entry.grid(row=13, column=1)
    
Train_nu_label = tk.Label(root, text="Train Number:").grid(row=11, column=0)
Train_nu_entry = tk.Entry(root)
Train_nu_entry.grid(row=11, column=1)

Ticket_type_label = tk.Label(root, text="Ticket Type:").grid(row=12, column=0)
Ticket_type_entry = tk.Entry(root)
Ticket_type_entry.grid(row=12, column=1)

result_label = tk.Label(root, text ='Result').grid(row=15, column=0)
result_text = tk.Text(root, height=10)
result_text.grid(row=16, column=1, columnspan=2)
result_count_text = tk.Label(root, text='')
result_count_text.grid(row=17, column=1, columnspan=2)

# Create the bottons to execute query when click and display results:

query1_button = tk.Button(root, text='Query 1', command=query1).grid(row=2, column=0, columnspan=2)

query2_button = tk.Button(root, text='Query 2', command=query2).grid(row=4, column=0, columnspan=2)

query3_button = tk.Button(root, text='Query 3', command=query3).grid(row=7, column=0, columnspan=2)

query4_button = tk.Button(root, text='Query 4', command=query4).grid(row=8, column=0, columnspan=2)

query5_button = tk.Button(root, text='Query 5', command=query5).grid(row=10, column=0, columnspan=2)

query6_button = tk.Button(root, text='Query 6', command=query6).grid(row=14, column=0, columnspan=2)


root.mainloop()
conn.close()